In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

# set dataframe outputs to three digits 
pd.set_option("display.precision", 3)
#suppress warnings
warnings.filterwarnings('ignore')

### Estimation models


M1: adaptation ~ perception + X * beta

M2: adaptation ~ awareness + X * beta

M3: adaptation ~ experience + X * beta

M4: adaptation ~ floodzone + X * beta 


beta = vector (age, homeownership, efficacy, insurance)

In [2]:
#import processed data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_k.csv'
df = pd.read_csv(path)
df.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'awareness', 'perception', 'experience', 'floodzone',
       'efficacy', 'supplies', 'insured', 'involved', 'learned_routes',
       'made_plan', 'made_safer', 'planned_neighbors', 'practiced_drills',
       'documents', 'rainy_day', 'alerts', 'family_communication', 'none',
       'dont_know', 'age', 'sex', 'education', 'race', 'homeownership',
       'income', 'rentmortgage', 'rurality', 'hazard_weight', 'geometry',
       'zip_count'],
      dtype='object')

In [3]:
# add hurricane dummy 
df['hurricane'] = np.nan
hurricane_state = 'Florida'
non_hurricane = ['Washington','Oregon','Pennsylvania']
df['hurricane'] = [1 if s in hurricane_state else 0 if s in non_hurricane else np.nan for s in df.state]

In [4]:
df = df.dropna()
df.shape

(261, 37)

In [5]:
def r_square(model):
    # McKelvay-Zavoina
    xb = model.predict(linear=True) #fitted latent value
    var_xb = np.var(xb,ddof=1) # variance of xb
    r2_mz = var_xb / (var_xb + 1) # McKelvay-Zavoina R_2
    # McFadden
    r2_mf = model.prsquared
    return r2_mz

In [6]:
def probit(functions, data):
    results_list = []
    for func in functions:
        model = smf.probit(formula=func, data=data).fit(disp=0)
        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,    
            'pseudoR_2': r_square(model),
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_marginal = model.get_margeff().summary_frame()
        df_model = pd.concat([df_model, df_marginal], axis =1)

        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results

In [7]:
#duplicate but with logit
def logit(functions, data):
    results_list = []
    for func in functions:
        model = smf.logit(formula=func, data=data).fit(disp=0)
        marg_effects = model.get_margeff().summary_frame()

        df_model = pd.DataFrame({
            'effect': model.params,               
            'p': model.pvalues,                   
            'marginal_effect': marg_effects['dy/dx'],
            'pseudoR_2': model.prsquared,
            'LLPr': model.llr_pvalue,
            'BIC': model.bic  
        })
        df_model.index = pd.MultiIndex.from_product([[func], df_model.index], names=['function', 'beta'])
        results_list.append(df_model)
    results = pd.concat(results_list)
    return results

In [8]:
# functions 
M1 = [
    'made_safer ~ hurricane + perception + age + homeownership',
    'documents ~ hurricane + perception + age + homeownership',
    'learned_routes ~ hurricane + perception + age + homeownership',
    'supplies ~ hurricane + perception + age + homeownership',
    'involved ~ hurricane + perception + age + homeownership',
    'made_plan ~ hurricane + perception + age + homeownership',
    'practiced_drills ~ hurricane + perception + age + homeownership',
    'alerts ~ hurricane + perception + age + homeownership',
    'family_communication ~ hurricane + perception + age + homeownership',
    'insured ~ hurricane + perception + age + homeownership'
]

M2 = [
    'made_safer ~ hurricane + awareness + age + homeownership',
    'documents ~ hurricane + awareness + age + homeownership',
    'learned_routes ~ hurricane + awareness + age + homeownership',
    'supplies ~ hurricane + awareness + age + homeownership',
    'involved ~ hurricane + awareness + age + homeownership',
    'made_plan ~ hurricane + awareness + age + homeownership',
    'practiced_drills ~ hurricane + awareness + age + homeownership',
    'alerts ~ hurricane + awareness + age + homeownership',
    'family_communication ~ hurricane + awareness + age + homeownership',
    'insured ~ hurricane + awareness + age + homeownership'

]

M3 = [
    'made_safer ~ hurricane + experience + age + homeownership',
    'documents ~ hurricane + experience + age + homeownership',
    'learned_routes ~ hurricane + experience + age + homeownership',
    'supplies ~ hurricane + experience + age + homeownership',
    'involved ~ hurricane + experience + age + homeownership',
    'made_plan ~ hurricane + experience + age + homeownership',
    'practiced_drills ~ hurricane + experience + age + homeownership',
    'alerts ~ hurricane + experience + age + homeownership',
    'family_communication ~ hurricane + experience + age + homeownership',
    'insured ~ hurricane + experience + age + homeownership'

]

M4 = [
    'made_safer ~ hurricane + floodzone + age + homeownership',
    'documents ~ hurricane + floodzone + age + homeownership',
    'learned_routes ~ hurricane + floodzone + age + homeownership',
    'supplies ~ hurricane + floodzone + age + homeownership',
    'involved ~ hurricane + floodzone + age + homeownership',
    'made_plan ~ hurricane + floodzone + age + homeownership',
    'practiced_drills ~ hurricane + floodzone + age + homeownership',
    'alerts ~ hurricane + floodzone + age + homeownership',
    'family_communication ~ hurricane + floodzone + age + homeownership',
    'insured ~ hurricane + floodzone + age + homeownership'

]

In [9]:
# probit regression
M1 = probit(functions = M1, data = df)
M2 = probit(functions = M2, data = df)
M3 = probit(functions = M3, data = df)
M4 = probit(functions = M4, data = df)

In [10]:
# store results 
M1, M2, M3, M4 = M1.round(3), M2.round(3), M3.round(3), M4.round(3)
M1.to_excel('results/M1.xlsx')
M2.to_excel('results/M2.xlsx')
M3.to_excel('results/M3.xlsx')
M4.to_excel('results/M4.xlsx')

### Sensitivity Check: Exclude dummy

In [11]:
# functions 
M1 = [
    'made_safer ~ perception + age + homeownership',
    'documents ~ perception + age + homeownership',
    'learned_routes ~ perception + age + homeownership',
    'supplies ~ perception + age + homeownership',
    'involved ~ perception + age + homeownership',
    'made_plan ~ perception + age + homeownership',
    'practiced_drills ~ perception + age + homeownership',
    'alerts ~ perception + age + homeownership',
    'family_communication ~ perception + age + homeownership',
    'insured ~ perception + age + homeownership'
]

M2 = [
    'made_safer ~ awareness + age + homeownership',
    'documents ~ awareness + age + homeownership',
    'learned_routes ~ awareness + age + homeownership',
    'supplies ~ awareness + age + homeownership',
    'involved ~ awareness + age + homeownership',
    'made_plan ~ awareness + age + homeownership',
    'practiced_drills ~ awareness + age + homeownership',
    'alerts ~ awareness + age + homeownership',
    'family_communication ~ awareness + age + homeownership',
    'insured ~ awareness + age + homeownership'
]

M3 = [
    'made_safer ~ experience + age + homeownership',
    'documents ~ experience + age + homeownership',
    'learned_routes ~ experience + age + homeownership',
    'supplies ~ experience + age + homeownership',
    'involved ~ experience + age + homeownership',
    'made_plan ~ experience + age + homeownership',
    'practiced_drills ~ experience + age + homeownership',
    'alerts ~ experience + age + homeownership',
    'family_communication ~ experience + age + homeownership',
    'insured ~ experience + age + homeownership'
]

M4 = [
    'made_safer ~ floodzone + age + homeownership',
    'documents ~ floodzone + age + homeownership',
    'learned_routes ~ floodzone + age + homeownership',
    'supplies ~ floodzone + age + homeownership',
    'involved ~ floodzone + age + homeownership',
    'made_plan ~ floodzone + age + homeownership',
    'practiced_drills ~ floodzone + age + homeownership',
    'alerts ~ floodzone + age + homeownership',
    'family_communication ~ floodzone + age + homeownership',
    'insured ~ floodzone + age + homeownership'
]

In [12]:
# probit regression
M1 = probit(functions = M1, data = df)
M2 = probit(functions = M2, data = df)
M3 = probit(functions = M3, data = df)
M4 = probit(functions = M4, data = df)

In [13]:
# store results 
M1, M2, M3, M4 = M1.round(3), M2.round(3), M3.round(3), M4.round(3)
M1.to_excel('results/M1_sensitivity.xlsx')
M2.to_excel('results/M2_sensitivity.xlsx')
M3.to_excel('results/M3_sensitivity.xlsx')
M4.to_excel('results/M4_sensitivity.xlsx')

### Now including insurance

In [14]:
# functions 
M1 = [
    'made_safer ~ hurricane + perception + age + homeownership + insured',
    'documents ~ hurricane + perception + age + homeownership + insured',
    'learned_routes ~ hurricane + perception + age + homeownership + insured',
    'supplies ~ hurricane + perception + age + homeownership + insured',
    'involved ~ hurricane + perception + age + homeownership + insured',
    'made_plan ~ hurricane + perception + age + homeownership + insured',
    'practiced_drills ~ hurricane + perception + age + homeownership + insured',
    'alerts ~ hurricane + perception + age + homeownership + insured',
    'family_communication ~ hurricane + perception + age + homeownership + insured',
]
M2 = [
    'made_safer ~ hurricane + awareness + age + homeownership + insured',
    'documents ~ hurricane + awareness + age + homeownership + insured',
    'learned_routes ~ hurricane + awareness + age + homeownership + insured',
    'supplies ~ hurricane + awareness + age + homeownership + insured',
    'involved ~ hurricane + awareness + age + homeownership + insured',
    'made_plan ~ hurricane + awareness + age + homeownership + insured',
    'practiced_drills ~ hurricane + awareness + age + homeownership + insured',
    'alerts ~ hurricane + awareness + age + homeownership + insured',
    'family_communication ~ hurricane + awareness + age + homeownership + insured',
]

M3 = [
    'made_safer ~ hurricane + experience + age + homeownership + insured',
    'documents ~ hurricane + experience + age + homeownership + insured',
    'learned_routes ~ hurricane + experience + age + homeownership + insured',
    'supplies ~ hurricane + experience + age + homeownership + insured',
    'involved ~ hurricane + experience + age + homeownership + insured',
    'made_plan ~ hurricane + experience + age + homeownership + insured',
    'practiced_drills ~ hurricane + experience + age + homeownership + insured',
    'alerts ~ hurricane + experience + age + homeownership + insured',
    'family_communication ~ hurricane + experience + age + homeownership + insured',
]

M4 = [
    'made_safer ~ hurricane + floodzone + age + homeownership + insured',
    'documents ~ hurricane + floodzone + age + homeownership + insured',
    'learned_routes ~ hurricane + floodzone + age + homeownership + insured',
    'supplies ~ hurricane + floodzone + age + homeownership + insured',
    'involved ~ hurricane + floodzone + age + homeownership + insured',
    'made_plan ~ hurricane + floodzone + age + homeownership + insured',
    'practiced_drills ~ hurricane + floodzone + age + homeownership + insured',
    'alerts ~ hurricane + floodzone + age + homeownership + insured',
    'family_communication ~ hurricane + floodzone + age + homeownership + insured',
]

In [15]:
# probit regression
M1 = probit(functions = M1, data = df)
M2 = probit(functions = M2, data = df)
M3 = probit(functions = M3, data = df)
M4 = probit(functions = M4, data = df)

In [16]:
# store results 
M1, M2, M3, M4 = M1.round(3), M2.round(3), M3.round(3), M4.round(3)
M1.to_excel('results/M1_sensitivity_insured.xlsx')
M2.to_excel('results/M2_sensitivity_insured.xlsx')
M3.to_excel('results/M3_sensitivity_insured.xlsx')
M4.to_excel('results/M4_sensitivity_insured.xlsx')